In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('AQE').getOrCreate()

In [4]:
spark

In [18]:
spark.conf.set("spark.sql.adaptive.enabled",True)

In [19]:
data = [
    ("Alice", "HR", 10000),
    ("Bob", "IT", 12000),
    ("Charlie", "Finance", 9000),
    ("David", "HR", 11000),
    ("Eve", "IT", 13000),
    ("Frank", "Finance", 8000)
]
columns = ["employee_name", "department", "salary"]
df = spark.createDataFrame(data, columns)

In [20]:
df.show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Alice|        HR| 10000|
|          Bob|        IT| 12000|
|      Charlie|   Finance|  9000|
|        David|        HR| 11000|
|          Eve|        IT| 13000|
|        Frank|   Finance|  8000|
+-------------+----------+------+



In [21]:
from pyspark.sql.functions import *

In [22]:
df =df.groupBy("department").agg(count("employee_name"))

In [24]:
df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[department#38], functions=[count(employee_name#37)])
   +- Exchange hashpartitioning(department#38, 200), ENSURE_REQUIREMENTS, [plan_id=72]
      +- HashAggregate(keys=[department#38], functions=[partial_count(employee_name#37)])
         +- Project [employee_name#37, department#38]
            +- Scan ExistingRDD[employee_name#37,department#38,salary#39L]




# **AQE JOIN Stratgy**

In [25]:
d1 =[
    (1,"Alice"),
    (2,"Bob"),
    (3,"Charlie"),
    (4,"David"),
    (5,"Eve"),
    (6,"Frank")
]
df1 = spark.createDataFrame(d1,["id","name"])
d2 = [
    (1,100000),
    (2,50000),
    (3,90000),
    (4,40000),
    (5,80000),
    (6,70000)
]
df2 = spark.createDataFrame(d2,["id","salary"])

In [26]:
df_new = df1.join(df2,df1['id'] == df2['id'],"left")

In [27]:
df_new.show()

+---+-------+---+------+
| id|   name| id|salary|
+---+-------+---+------+
|  1|  Alice|  1|100000|
|  3|Charlie|  3| 90000|
|  2|    Bob|  2| 50000|
|  6|  Frank|  6| 70000|
|  5|    Eve|  5| 80000|
|  4|  David|  4| 40000|
+---+-------+---+------+



In [29]:
df_new.explain(True)

== Parsed Logical Plan ==
Join LeftOuter, (id#65L = id#69L)
:- LogicalRDD [id#65L, name#66], false
+- LogicalRDD [id#69L, salary#70L], false

== Analyzed Logical Plan ==
id: bigint, name: string, id: bigint, salary: bigint
Join LeftOuter, (id#65L = id#69L)
:- LogicalRDD [id#65L, name#66], false
+- LogicalRDD [id#69L, salary#70L], false

== Optimized Logical Plan ==
Join LeftOuter, (id#65L = id#69L)
:- LogicalRDD [id#65L, name#66], false
+- Filter isnotnull(id#69L)
   +- LogicalRDD [id#69L, salary#70L], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [id#65L], [id#69L], LeftOuter
   :- Sort [id#65L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(id#65L, 200), ENSURE_REQUIREMENTS, [plan_id=219]
   :     +- Scan ExistingRDD[id#65L,name#66]
   +- Sort [id#69L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#69L, 200), ENSURE_REQUIREMENTS, [plan_id=220]
         +- Filter isnotnull(id#69L)
            +- Scan ExistingRDD[id#69L,s